In [89]:
# Create Sagemaker execution role
import pickle
import numpy as np
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.tensorfl import SKLearn
from sklearn.metrics import accuracy_score
import boto3
import os

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()

s3_prefix = "image-recognition-script-mode-workflow"
pickle_s3_prefix = f"{s3_prefix}/pickle"
pickle_s3_uri = f"s3://{bucket}/{s3_prefix}/pickle"
pickle_train_s3_uri = f"{pickle_s3_uri}/train"
pickle_test_s3_uri = f"{pickle_s3_uri}/test"

train_dir = os.path.join(os.getcwd(), "")
test_dir = os.path.join(os.getcwd(), "")

In [90]:
def pickleTrainingData():
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict

    train_data = np.empty((0, 32*32*3))
    train_labels = []

    for i in range(1, 2):
        fileNameDataBatch = './cifar-10-batches-py/data_batch_' + str(i)
        batch = unpickle(fileNameDataBatch)
        train_data = np.vstack((train_data, batch[b'data']))
        train_labels += batch[b'labels']

    train_labels = np.array(train_labels)
    train_data = train_data.reshape(-1, 32, 32, 3) / 255.0

    # !!!!! NOTICE HOW THE DATA IS SAVED !!!!!!
    # Will be returned in form of:
    # train_label, train_data  = getDataBack()
    pickle.dump([train_labels, train_data], open('./train.cnn', 'wb'))


pickleTrainingData()

In [91]:
def getTestData():
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict
    fileNameTestBatch = './cifar-10-batches-py/test_batch'
    test_data = unpickle(fileNameTestBatch)[b'data']
    test_data = test_data.reshape(-1, 32, 32, 3) / 255.0
    test_labels = np.array(unpickle(fileNameTestBatch)[b'labels'])

    num_samples_to_select = 600
    random_indices = np.random.choice(test_data.shape[0], num_samples_to_select, replace=False)
    selected_test_data = test_data[random_indices]
    selected_test_labels = test_labels[random_indices]
    pickle.dump([selected_test_labels, selected_test_data], open('./test_batch', 'wb'))


getTestData()

In [92]:
s3_resource_bucket = boto3.Session().resource("s3").Bucket(bucket)
s3_resource_bucket.Object(os.path.join(pickle_s3_prefix, "train.cnn")).upload_file(
    train_dir + "/train.cnn"
)
s3_resource_bucket.Object(os.path.join(pickle_s3_prefix, "test_batch")).upload_file(
    test_dir + "/test_batch"
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [93]:
# This is not required as these values are the defaults:
hyperparameters = {
    "copy_X": True,
    "fit_intercept": True,
    "normalize": False,
}

In [94]:
entry_point = "cnnScript.py"

# Modify this based on your instance type / size
train_instance_type = "ml.m5.large"

estimator_parameters = {
    "entry_point": entry_point,
    "source_dir": "script",
    "framework_version": "0.23-1",
    "py_version": "py3",
    "instance_type": train_instance_type,
    "instance_count": 1,
    "hyperparameters": hyperparameters,
    "role": role,
    "base_job_name": "cnn-model",
}

estimator = SKLearn(**estimator_parameters)

In [95]:
inputs = {
    "train": pickle_train_s3_uri,
    "test": pickle_test_s3_uri
}

# starting the training job
estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: cnn-model-2024-11-14-05-26-57-146


2024-11-14 05:26:59 Starting - Starting the training job...
2024-11-14 05:27:13 Starting - Preparing the instances for training...
2024-11-14 05:27:37 Downloading - Downloading input data...
2024-11-14 05:28:07 Downloading - Downloading the training image...
2024-11-14 05:28:43 Training - Training image download completed. Training in progress.2024-11-14 05:28:47,681 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-11-14 05:28:47,685 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-14 05:28:47,726 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-11-14 05:28:47,890 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-14 05:28:47,901 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-14 05:28:47,912 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-14 05:2

In [96]:
tensorflow_predictor = estimator.deploy(initial_instance_count=1,
                                        instance_type='ml.m5.large',
                                        endpoint_name='cnn-endpoint4')

INFO:sagemaker:Creating model with name: cnn-model-2024-11-14-05-30-39-861
INFO:sagemaker:Creating endpoint-config with name cnn-endpoint4
INFO:sagemaker:Creating endpoint with name cnn-endpoint4


-------!

In [97]:
# Get Accuracy result
def getTestDataBack(path):
    # Find all files with a cnn ext but we only load the first one in this sample:
    files = [os.path.join(path, file) for file in os.listdir(path) if file == "test_batch"]

    if len(files) == 0:
        raise ValueError("Invalid # of files in dir: {}".format(path))

    [X, y] = pickle.load(open(files[0], 'rb'))
    return X, y


def getAccuracyOfPrediction(cnn_predictions, test_labels):
    cnn_predicted_labels = np.argmax(cnn_predictions, axis=1)
    accuracy = accuracy_score(test_labels, cnn_predicted_labels)
    print("Accuracy:", accuracy)


test_labels, test_data = getTestDataBack(".")
cnn_predictions = tensorflow_predictor.predict(test_data)

getAccuracyOfPrediction(cnn_predictions, test_labels)

Accuracy: 0.485


In [ ]:
tensorflow_predictor.delete_endpoint(True)